In [1]:
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
from nltk.metrics import ConfusionMatrix
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.externals import joblib
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [2]:
emo=5

if emo == 2: 
    dataset_path = '../Data/Datasets/Binary Classification/'
else:
    dataset_path = '../Data/Datasets/Multiclass Classification/'

In [3]:
#build the classifier
train = pd.read_csv(dataset_path+'train.csv', sep=",", header=None,index_col = False)
test = pd.read_csv(dataset_path+'test.csv', sep=",", header=None,index_col = False)

whole_ds = train.append(test,ignore_index=True )
whole_ds['tokenized_sents'] = whole_ds.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)

onehot_enc = MultiLabelBinarizer()
onehot_enc.fit(whole_ds['tokenized_sents'])


MultiLabelBinarizer(classes=None, sparse_output=False)

In [4]:
#hold out testing
train = pd.read_csv(dataset_path+'train.csv', sep=",", header=None,index_col = False)
test = pd.read_csv(dataset_path+'test.csv', sep=",", header=None,index_col = False)


train['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in train.iterrows()]
test['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in test.iterrows()]
 
nb = GaussianNB()
nb.fit(onehot_enc.transform(train['tokenized_sents']), train[1])
score = nb.score(onehot_enc.transform(test['tokenized_sents']), test[1])
print("Holdout testing accuracy: {}".format(score))

Holdout testing accuracy: 0.6371398078975453


In [5]:
#Cross validation testing
accuracy_gnb = []
folds = 10

for idx in range(1,folds+1):
    
    train = pd.read_csv(dataset_path+'cv'+str(idx)+'_train.csv', sep=",", header=None,index_col = False)
    test = pd.read_csv(dataset_path+'cv'+str(idx)+'_test.csv', sep=",", header=None,index_col = False)
    
    train['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in train.iterrows()]
    test['tokenized_sents'] = [nltk.word_tokenize(row[0]) for index, row in test.iterrows()]
 
    
    nb = GaussianNB()
    nb.fit(onehot_enc.transform(train['tokenized_sents']), train[1])
    score = nb.score(onehot_enc.transform(test['tokenized_sents']), test[1])
    accuracy_gnb.append(score)
    print("Fold: {}: accuracy = {}".format(idx,score))

print("10 fold Cross Validation accuracy = {}".format(np.mean(accuracy_gnb)))

Fold: 1: accuracy = 0.6878306878306878
Fold: 2: accuracy = 0.5941644562334217
Fold: 3: accuracy = 0.6223404255319149
Fold: 4: accuracy = 0.5904255319148937
Fold: 5: accuracy = 0.5614973262032086
Fold: 6: accuracy = 0.4932975871313673
Fold: 7: accuracy = 0.5924932975871313
Fold: 8: accuracy = 0.5683646112600537
Fold: 9: accuracy = 0.5308310991957105
Fold: 10: accuracy = 0.6219839142091153
10 fold Cross Validation accuracy = 0.5863228937097504
